In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder ,StandardScaler
import joblib
from pathlib import Path
from ydata_profiling import ProfileReport
import os

In [2]:
df = pd.read_csv("../data/processed/cleaned_v1.csv")
df1 = pd.read_csv("../data/processed/test_v1.csv")
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28,0,> 2 Years,Yes,40454,26,217,1
1,2,Male,76,1,3,0,1-2 Year,No,33536,26,183,0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294,26,27,1
3,4,Male,21,1,11,1,< 1 Year,No,28619,152,203,0
4,5,Female,29,1,41,1,< 1 Year,No,27496,152,39,0


In [3]:
#Cleaning column names
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df1.columns = df1.columns.str.strip().str.replace(' ', '_').str.lower()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369800 entries, 0 to 369799
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    369800 non-null  int64 
 1   gender                369800 non-null  object
 2   age                   369800 non-null  int64 
 3   driving_license       369800 non-null  int64 
 4   region_code           369800 non-null  int64 
 5   previously_insured    369800 non-null  int64 
 6   vehicle_age           369800 non-null  object
 7   vehicle_damage        369800 non-null  object
 8   annual_premium        369800 non-null  int64 
 9   policy_sales_channel  369800 non-null  int64 
 10  vintage               369800 non-null  int64 
 11  response              369800 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 33.9+ MB


In [5]:
#encoding data
# #Converting objects to numeric
# df['Gender'] = df['Gender'].map({'Male':0, 'Female':1})
# df['Vehicle_Damage'] = df['Vehicle_Damage'].map({"Yes":1, "No":0})

# #Converting vehicle age to ordered values
# df["Vehicle_Age"] = df["Vehicle_Age"].map({
#     '< 1 Year' : 0,
#     '1-2 Year':1,
#     '> 2 Years':2 
# })
label_enc_cols = ['gender', 'vehicle_age', 'vehicle_damage']
label_encoders = {}

for col in label_enc_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    
    
label_enc_cols = ['gender', 'vehicle_age', 'vehicle_damage']
label_encoders = {}

for col in label_enc_cols:
    le = LabelEncoder()
    df1[col] = le.fit_transform(df1[col])
    label_encoders[col] = le

In [6]:
# import dtale
# import pandas as pd
# import os
# from pathlib import Path

# # Load your dataset
# df = pd.read_csv("E:/PROJECTS/insurance_fraud_detection/data/raw/train.csv")

# # Start D-Tale and print the URL to access
# d = dtale.show(df, name='Insurancedataset', open_browser=True)


In [7]:
#Checking for class imbalance
df["response"].value_counts(normalize=True)

response
0    0.878207
1    0.121793
Name: proportion, dtype: float64

In [8]:
#Making class balance
from imblearn.over_sampling import SMOTE

X = df.drop('response', axis=1)
y = df['response']

smote = SMOTE(random_state = 42)
X_resampled, y_resampled = smote.fit_resample(X,y)

print(y_resampled.value_counts(normalize=True))

df_resampled = pd.concat([X_resampled, y_resampled], axis = 1)

#Data balanced

response
1    0.5
0    0.5
Name: proportion, dtype: float64


In [9]:
df = df_resampled

In [10]:
df = df.rename(columns = {
    'response' : 'is_fraud',
})

In [11]:
#Finding Correlation between them

corr_matrix = df.corr()

print('Correlation with target: ')
print(corr_matrix['is_fraud'].sort_values(ascending=False))

Correlation with target: 
is_fraud                1.000000
vehicle_damage          0.570616
age                     0.178639
annual_premium          0.026727
driving_license         0.005929
region_code             0.003042
id                     -0.003475
vintage                -0.004932
gender                 -0.126885
policy_sales_channel   -0.216907
vehicle_age            -0.342255
previously_insured     -0.594456
Name: is_fraud, dtype: float64


In [12]:
#Feature Selection
df = df.drop(columns=['id','region_code','vintage','driving_license','annual_premium'])
df1 = df1.drop(columns=['id','region_code','vintage','driving_license','annual_premium'])

In [13]:
df.head(10)

,gender,age,previously_insured,vehicle_age,vehicle_damage,policy_sales_channel,is_fraud
0,1,44,0,2,1,26,1
1,1,76,0,0,0,26,0
2,1,47,0,2,1,26,1
3,1,21,1,1,0,152,0
4,0,29,1,1,0,152,0
5,0,24,0,1,1,160,0
6,1,23,0,1,1,152,0
7,0,56,0,0,1,26,1
8,0,24,1,1,0,152,0
9,0,32,1,1,0,152,0


In [14]:
# from ydata_profiling import ProfileReport
# import pandas as pd
# import os
# from pathlib import Path

# # Disable chi-squared threshold (fixes the AttributeError)
# from ydata_profiling.config import Settings
# config = Settings()
# config.vars.num.chi_squared_threshold = 0 

# # Create report path
# profile_path = os.path.abspath("../reports/eda_report.html")
# Path(os.path.dirname(profile_path)).mkdir(parents=True, exist_ok=True)

# # Create profiling report
# profile = ProfileReport(
#     df,
#     title="Insurance Fraud Detection - EDA Report",
#     explorative=True,
#     config=config
# )

# # Export to file
# profile.to_file(profile_path)
# print("Profiling Completed")

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649522 entries, 0 to 649521
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   gender                649522 non-null  int32
 1   age                   649522 non-null  int64
 2   previously_insured    649522 non-null  int64
 3   vehicle_age           649522 non-null  int32
 4   vehicle_damage        649522 non-null  int32
 5   policy_sales_channel  649522 non-null  int64
 6   is_fraud              649522 non-null  int64
dtypes: int32(3), int64(4)
memory usage: 27.3 MB


In [16]:
df.nunique()

gender                    2
age                      66
previously_insured        2
vehicle_age               3
vehicle_damage            2
policy_sales_channel    163
is_fraud                  2
dtype: int64

In [17]:
binary_cols = ["gender", "previously_insured", "vehicle_age", "vehicle_damage"]
target_col = "is_fraud"

X_num = df.drop(columns=binary_cols + [target_col])
X_bin = df[binary_cols]
y = df[target_col]

scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)
df_scaled = pd.concat([X_num_scaled, X_bin.reset_index(drop=True), y.reset_index(drop=True)], axis=1)

# Save to CSV
processed_path = "../data/labeled"
Path(processed_path).mkdir(parents=True, exist_ok=True)
df_scaled.to_csv(f"{processed_path}/labeled_data.csv", index=False)
model_path = "../models"
Path(model_path).mkdir(parents=True, exist_ok=True)
joblib.dump(scaler, f"{model_path}/scaler.pkl")
print("Model Saved.")

Model Saved.


In [18]:
# Load test data
test_df = df1

# Separate numeric and binary columns
X_test_num = test_df.drop(columns=binary_cols, errors='ignore')
X_test_bin = test_df[binary_cols]

# Load saved scaler
scaler = joblib.load("../models/scaler.pkl")

# Apply same transformation
X_test_num_scaled = scaler.transform(X_test_num)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns)

# Combine
X_test_final = pd.concat([X_test_num_scaled, X_test_bin.reset_index(drop=True)], axis=1)

# Save final test features
X_test_final.to_csv("../data/labeled/labeled_test_data.csv", index=False)
print("Test data transformed and saved.")


Test data transformed and saved.
